In [45]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras 
from keras import Sequential
from keras.layers import Dense
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
df = pd.read_csv("/path-to-your-file.csv")

In [47]:
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [48]:
df.corr().Outcome

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [49]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [50]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [51]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
scalar = StandardScaler()

In [54]:
X = scalar.fit_transform(X)

In [55]:
X[0]

array([ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
        0.20401277,  0.46849198,  1.4259954 ])

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.33, random_state=42 ) 

In [57]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [58]:
model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
33/33 [==============================] - 1s 6ms/step - loss: 0.7241 - accuracy: 0.4611 - val_loss: 0.6694 - val_accuracy: 0.5945
Epoch 2/15
33/33 [==============================] - 0s 2ms/step - loss: 0.6461 - accuracy: 0.6615 - val_loss: 0.6143 - val_accuracy: 0.7205
Epoch 3/15
33/33 [==============================] - 0s 2ms/step - loss: 0.5900 - accuracy: 0.7296 - val_loss: 0.5743 - val_accuracy: 0.7441
Epoch 4/15
33/33 [==============================] - 0s 2ms/step - loss: 0.5470 - accuracy: 0.7588 - val_loss: 0.5454 - val_accuracy: 0.7402
Epoch 5/15
33/33 [==============================] - 0s 3ms/step - loss: 0.5171 - accuracy: 0.7724 - val_loss: 0.5296 - val_accuracy: 0.7323
Epoch 6/15
33/33 [==============================] - 0s 2ms/step - loss: 0.4939 - accuracy: 0.7665 - val_loss: 0.5201 - val_accuracy: 0.7362
Epoch 7/15
33/33 [==============================] - 0s 2ms/step - loss: 0.4774 - accuracy: 0.7743 - val_loss: 0.5146 - val_accuracy: 0.7441
Epoch 8/15
33/33 [==

**Now let's try it with hyper_parameter tuning**

In [59]:
import kerastuner as kt

In [64]:
from keras.engine.training import optimizers
def build_model(hp):
    model = keras.Sequential()
    
    model.add(Dense(30, activation='relu', input_dim=8))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer=hp.Choice('optimizer',['Adam','RMSprop']), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [65]:
tuner_ = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 4,
    directory="my_dir",
    project_name="helloworld",
)

In [66]:
tuner_.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [67]:
tuner_.get_best_hyperparameters()[0].values

{'optimizer': 'RMSprop'}